# <center>제1고지 미분 자동 계산</center>

# 1단계 상자로서의 변수

변수의 성질  
- 변수와 데이터는 별개다.
- 변수에는 데이터가 들어간다(대입 혹은 할당한다).
- 변수 속을 들여다보면 데이터를 알 수 있다(참조한다).

데이터를 담는 변수(variable) 클래스

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


여기서 x는 Variable의 인스턴스  
x는 데이터 자체가 아니라 데이터를 담은 상자  

In [3]:
x.data = np.array(2.0)
print(x.data)

2.0


<img src='./img/1/dimension.png' width=500>

넘파이의 다차원 배열의 예  
  
다차원 배열에서 원소의 순서에는 방향이 있고, 이 방향을 차원(dimension) 혹은 축(axis)라고 함  
위 그림의 왼쪽부터 0차원 배열, 1차원 배열, 2차원 배열이며,  
차례대로 스칼라(scalar), 벡터(vector), 행렬(matrix)라고 함  
또한, 다차원 배열을 텐서(tensor)라고도 함  
  
넘파이의 ndim(number fo dimension)이라는 인스턴스 변수를 사용하여 다차원 배열의 차원 수를 확인할 수 있음

In [4]:
import numpy as  np
x = np.array(1)
print(x.ndim)

x = np.array([1,2,3])
print(x.ndim)

x = np.array([[1,2,3],[4,5,6]])
print(x.ndim)

0
1
2


- 벡터의 차원 : 벡터의 원소 수   
- 배열의 차원 : 축의 수

# 2단계 변수를 낳는 함수

<img src='./img/1/function_1.png' width=500>  
  
함수 : 어떤 변수로부터 다른 변수로의 대응 관계를 정한 것  
  
위 그림처럼 노드(node)와 에지(edge)로 계산 과정을 표현한 그림을 계산 그래프(computational graph)라고 함

앞서 구현한 Variable 인스턴스를 변수로 다룰 수 있는 함수를 Function 클래스로 구현  
- Function 클래스는 Variable 인스턴스를 입력받아 Variable 인스턴스를 출력함 
- Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 있음  

In [5]:
class Function:
    def __call__(self, input): # input은 Variable 인스턴스
        x = input.data # 데이터를 꺼낸다.
        y = x ** 2 # 실제 계산
        output = Variable(y) # Variable 형태로 되돌린다.
        return output

__call__ 메서드는 파이썬의 특수 메서드로  
이 메서드를 정의하면 f=Function() 형태로 함수의 인스턴스를 변수 f에 대입해두고  
나중에 f(...)의 형태로 \_\_call__ 메서드를 호출할 수 있음

In [6]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


Function 클래스를 기반 클래스로 두고 DeZero의 모든 함수가 공통적으로 제공하는 기능만 담아두도록 수정  
- Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현  
- 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현  

In [7]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError()

\_\_call__ 메서드는 'Variable에서 데이터 찾기'와 '계산 결과를 Variable에 포장하기' 수행  
구체적인 계산은 forward 메서드를 호출하여 수행

In [8]:
class Square(Function):
    def forward(self, x):
        return x ** 2

square 클래스는 Function 클래스를 상속받기 때문에 \_\_call__ 메서드는 그대로 계승됨

In [9]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# 3단계 함수 연결

$y=e^x$를 계산하는 함수 추가  
여기서 e는 자연로그의 밑으로 구체적인 값은 2.718...(오일러의 수 혹은 네이피어 상수라고 함)

In [10]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

함수 연이어 사용하기(합성 함수(composite function)  
Function 클래스의 \_\_call__ 메서드의 입출력이 Variable 인스턴스로 통일되어 있어서 여러 함수를 연속하여 적용할 수 있음
  
<img src='./img/1/function_2.png' width=800>
$y=(e^{x^2})^2$ 계산

In [11]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


# 4단계 수치 미분

미분이란 '극한으로 짧은 시간(순간)'에서의 변화량을 뜻함  
수식으로 표현하면 다음과 같다.  
  
$f'(x)=lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}$  
  
$x$와 $x+h$라는 두 점에서 함수 $f(x)$의 변화 비율은 $lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}$  
여기서 폭 $h$를 한없이 0에 가깝게 줄여 $x$의 변화율을 구하면 그 것이 미분 값이 됨  
$y=f(x)$가 어떤 구간에서 미분 가능하다면 해당 구간의 '모든 x'에서 성립  
$f'(x)$는 $f(x)$의 도함수라고 함

컴퓨터는 극한을 취급할 수 없어 h를 근한화 비슷한 매우 작은 값을 이용  
이런 미세한 차이를 이용하여 함수의 변화량을 구하는 방법을 __수치 미분(numerical differentiation)__이라고 함  
  
수치 미분은 작은 값을 사용하여 '진정한 미분'을 근사함  
어쩔 수 없이 오차가 포함되는데, 이 오차를 줄이는 방법으로 '중앙차분(centered difference)'이 있음  
  
중앙차분은 $f(x)$와 $f(x+h)$의 차이를 구하는 대신 $f(x-h)$와 $f(x+h)$의 차이를 구함  
($f(x)$와 $f(x+h)$ 지점에서의 기울기를 구하는 방법은 '전진차분(forward difference)'라고 함)  
중앙차분은 전진차분에 비해 상대적으로 오차가 적음  
  
중앙차분에서 직선의 기울기는 $\frac{f(x+h)-f(x-h)}{2h}$가 된다.  
  
중앙차분을 이용하여 수치 미분을 계산하는 함수 구현  
- 인수목록  
    * f : 미분의 대상이 되는 함수이자 Function의 인스턴스
    * x : 미분을 계산하는 변수
    * eps : epsilon(엡실론)의 약어로, 작은 값을 나타냄

In [12]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

Square 클래스를 대상으로 미분

In [13]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


< 참고 >  
  
해석적으로 계산하는 방법 : 수식 변형만으로 답을 유도함  
따라서 오차를 포함하지 않는 정확한 값

$y=(e^{x^2})^2$ 미분

In [14]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


파이썬에서는 함수도 객체이기 때문에 다른 함수에 인수로 전달할 수 있음

하지만 수치 미분은 자릿수 누락이 발생하여 유효 자릿수가 줄어들 수 있고, 따라서 오차가 포함되기 쉬움  
또한, 수치 미분은 계산량이 많음  
  
따라서 역전파를 사용하는데,  
수치 미분은 구현하기 쉽고 거의 정확한 값을 얻을 수 있는 반면  
역전파는 복잡한 알고리즘으로 구현 중 버그가 섞여 들어가기 쉬움  
그래서 역전파를 정확하게 구현했는지를 확인하기 위해 수치 미분의 결과를 이용하곤 함  
이를 기울기 확인(gradient checking)이라고 함  

# 5단계 역전파 이론

역전파(backpropagation, 오차역전파법)을 이용하면 미분을 효율적으로 계산할 수 있고 결괏값의 오차도 더 작음  
  
연쇄 법칙(chain rule) : 합성 함수(여러 함수가 연결된 함수)의 미분은 구성 함수 각각의 미분을 곱한 것과 같다.  
  
<img src='./img/1/function_2.png' width=800>  
  
위 그림처럼 $y=F(x)$라는 함수는 $a=A(x)$, $b=B(a)$, $y=C(b)$라는 세 함수로 구성되어있다고 할 때, 
x에 대한 y의 미분은 다음과 같다.  
  
$\frac{\partial{y}}{\partial{x}}=\frac{\partial{y}}{\partial{b}}\frac{\partial{b}}{\partial{a}}\frac{\partial{a}}{\partial{x}}$  
  
x에 대한 y의 미분은 구성 함수 각각의 미분값을 모두 곱한 값과 같기 때문에,  
합성 함수의 미분은 각 함수의 국소적인 미분들로 분해할 수 있다.  
  
계산의 흐름은 출력에서 입력 방향으로, 보토으이 계산과는 반대 방향으로 미분을 계산함  
역전파 계산 그래프를 예를 들어 $A'(x)$의 곱셈을 $A'(x)$노드로 간략하게 표현하면 다음과 같다.  
  
<img src='./img/1/backpropagation.png' width=800>  
  
전파되는 데이터는 모두 'y의 미분값'임  
머신러닝은 주로 대량의 매개변수를 입력받고 마지막에 손실 함수(loss function)을 거쳐 출력을 내는 형태로 진행되는데,  
손실 함수의 출력은 '중요 요소'로, 손실 함수의 각 매개변수에 대한 미분을 계산해야 함  
이런 경우 미분값을 출력에서 입력 방향으로 전파하면 한 번의 전파만으로 모든 매개변수에 대한 미분을 계산할 수 있음  
따라서 계산이 효율적으로 이뤄지는 반대 방향으로 전파하는 방식인 역전파를 이용함  
  
역전파에서는 순전파 시 이용한 데이터가 필요하므로  
먼저 순전파를 하고, 이때 각 함수가 입력 변수의 값을 기억해둬야 함

# 6단계 수동 역전파

역전파에 대응하는 Variable 클래스를 구현하기 위해 통상값(data)와 더불어 그에 대응하는 미분값(grad)도 저장하도록 확장  
  
< 참고 > 벡터나 행렬 등 다변수에 대한 미분은 기울기(gradient)라고 함

In [15]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

Function 클래스에 두 기능을 추가  
- 미분을 계산하는 역전파(backward 메서드)  
    backward 메서드의 인수 중 dy는 출력 쪽에서 전해지는 미분값을 전달하는 역할  
- forward 메서드 호출 시 건네받은 Variable 인스턴스 유지  

In [16]:
class Function:
    def __call__(self, input):
        x = input.data
        y =self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억(보관)
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()

Square 클래스 추가 구현

In [24]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

Exp 클래스 추가 구현  
  
$y=e^x$의 미분은 $\frac{\partial{y}}{\partial{x}}=e^x$

In [25]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

<img src='./img/1/forward_ex1.png' width=800>

위 그림의 계산을 순전파

In [19]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

<img src='./img/1/backward_ex1.png' width=800>

역전파 코드

In [20]:
y.grad = np.array(1.0) # 자기 자신의 미분은 1
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


4단계에서 수치 미분으로 구한 값 : 3.2974426293330694  
수치 미분으로 구한 값과 역전파로 구한 값이 거의 같음

# 7단계 역전파 자동화

Define-bt-Run이란 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식으로, '동적 계산 그래프'라고도 함  
  
일반적인 계산(순전파)을 한 번만 해주면 역전파가 자동으로 이뤄지는 구조로 자동화  
  
함수의 순서를 리스트 형태로 저장해두면 나중에 거꾸로 추적하는 식으로 역전파를 자동화할 수 있음  
리스트 데이터 구조를 응용하면 수행한 계산을 리스트에 추가해 나가는 것만으로 어떠한 계산 그래프의 역전파도 제대로 해낼 수 있음! 이 데이터 구조를 웬거트 리스트(Wengert List) 혹은 테이프(tape)라고 함  
  
<img src='./img/1/variable_function_1.png' width=600>  
  
변수와 함수의 '관계'를 보면,  
함수 입장에서 변수는 '입력'과 '출력'에 쓰이고,  
변수 입장에서 함수는 '창조자' 혹은 '부모'다.  
창조자인 함수가 존재하지 않는 변수는 사용자에 의해 만들어진 변수로 간주됨  
  
이 관계를 맺어주도록 Variable 클래스와 Function 클래스를 수정

In [21]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func # 창조자 설정

In [22]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자를 설정
        self.input = input
        self.output = output # 출력 저장
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()

순전파를 계산하면 그 결과로 output이라는 Variable 인스턴스가 생성되고,  
이 때 생성된 output에 '창조자'를 기억시킴  
이 연결이 동적으로 만드는 기법의 핵심!  
이를 통해 계산 그래프를 거꾸로 올라갈 수 있음

아래 코드 실행 전 Square 클래스와 Exp 클래스 다시 실행해주기(수정된 Function 클래스 상속받도록 적용)

In [26]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

여기서, assert 는 결과가 True가 아니면 예외가 발생함  
따라서 assert 문은 조건을 충족하는지 여부를 확인하는데 사용할 수 있음  
  
<img src='./img/1/variable_function_2.png' width=600>  

함수와 변수 사이의 연결은 실제로 계산을 수행하는 시점에(순전파로 데이터를 흘려보낼 때) 만들어짐  
이러한 특성에 이름을 붙인 것이 Define-by-Run  
데이터를 흘려보냄으로써(Run) 연결이 규정(Define)된다는 뜻  
  
또한 위 그림과 같이 노드들의 연결로 이루어진 데이터 구조를 '링크드 리스트(linked list)'라고 함  
노드는 그래프를 구성하는 요소를,  
링크는 다른 노드를 가리키는 참조를 뜻함  
  
변수와 함수의 관계를 이용하여 역전파 구현  
순서는 다음과 같다.  
1. 함수를 가져온다.
2. 함수의 입력을 가져온다.
3. 함수의 backward 메서드를 호출한다.

In [27]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져옴
b = C.input # 2. 함수의 입력을 가져옴
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출

B = b.creator
a = B.input
a.grad = B.backward(b.grad)
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


위에서 구현한 반복 작업을 자동화할 수 있도록 Variable 클래스에 backward라는 새로운 메서드 추가

In [28]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # 1. 함수를 가져옴
        if f is not None:
            x = f.input # 2. 함수의 입력을 가져옴
            x.grad = f.backward(self.grad) # 3. 함수의 backward 메서드를 호출
            x.backward() # 하나 앞 변수의 backward 메서드를 호출(재귀)            

In [29]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 8단계 재귀에서 반복문으로

위에서 구현한 Variable의 backward 메서드는 self.creator가 None인 변수가 나올 때까지  
backward 메서드에서 backward 메서드를 호출하고, 호출된 backward 메서드에서 또다른 backward 메서드를 호출하는 과정이 반복됨  
이러한 구조를 재귀라고 함  
  
재귀를 사용한 구현에서 반복문을 이용한 구현 수정  

In [30]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져옴
            x, y = f.input, f.output # 함수의 입력과 출력을 가져옴
            x.grad = f.backward(y.grad) # backward 메서드를 호출
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가

In [31]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


재귀는 함수를 호출할 때마다 중간 결과를 메모리에 유지하면서(스택에 쌓으면서) 처리를 이어가기 때문에 일반적으로 반복문 방식이 처리 효율이 더 좋음

# 9단계 함수를 더 편리하게

### 파이썬 함수로 이용하기  
  
지금까지 구현한건 Square 클래스를 사용하는 계산을 하기 위해 다음과 같이 3줄의 코드가 필요함  
~~~python
x = Variable(np.array(0.5))
f = Square()
y = f(x)
~~~

'파이썬 함수'로 구현하면 더 간단하게 구현할 수 있음  

In [32]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

더 줄여서 한 줄로 구현할 수 있음

In [33]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [34]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [35]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### backward 메서드 간소화

역전파할 때마다 작성했던
~~~python
y.grad = np.array(1.0)
~~~
이 부분 생략하기 위해 Variable의 backward 메서드 수정

In [36]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # grad가 None이면 self.data와 같은 형상과 데이터 타입으로 1로 채워서 돌려줌
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [37]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.backward()
print(x.grad)

3.297442541400256


DeZero의 Variable은 데이터로 ndarry 인스턴스만 취급하게끔 의도했기 때문에,  
Variable에 ndarray 인스턴스 외의 데이터를 넣을 경우 즉시 오류를 일으키도록 하여  
문제를 조기에 발견할 수 있게끔 수정

In [38]:
class Variable:
    def __init__(self, data):
        if data is not None: # 데이터가 None이 아니고
            if not isinstance(data, np.ndarray): # ndarray 인스턴스도 아니면
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) # TypeError라는 예외 발생
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)

In [39]:
x = Variable(np.array(1.0))

In [40]:
x = Variable(None)

In [41]:
x = Variable(1.0) # 오류 발생

TypeError: <class 'float'>은(는) 지원하지 않습니다.

하지만 위에서 수정한 부분에는 문제가 있음

In [42]:
x = np.array([1.0]) # 1차원 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y)) # 1차원 ndarray에 제곱을 해도 ndarray

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [43]:
x = np.array(1.0) # 0차원 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y)) # 0차원 ndarray에 제곱을 하면 np.float64가 됨 

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


0차원 ndarray 인스턴스를 사용하여 계산하면 데이터 타입이 numpy.float64나 numpy.float32 등으로 달라짐  

np.isscalar는 입력 데이터가 numpy.float64 같은 스칼라 타입인지 확인해주는 함수  
(파이썬의 int와 float 타입도 스칼라로 판단)  

In [44]:
import numpy as np
np.isscalar(np.float64(1.0))

True

In [45]:
np.isscalar(2.0)

True

In [46]:
np.isscalar(np.array(1.0))

False

In [47]:
np.isscalar(np.array([1,2,3]))

False

위 예시처럼 x가 스칼라 타입인지 쉽게 확인할 수 있으며,  
이를 이용하여 입력이 스칼라인 경우 ndarray 인스턴스로 변환해주는 함수 구현

In [48]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

as_array 함수를 Function 클래스에 추가

In [49]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) # as_array 함수 적용
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()

# 10단계 테스트

파이썬으로 테스트할 때는 표준 라이브러리에 포함된 unittest를 사용할 수 있음  

unittest.TestCase를 상속한 SquareTest 클래스 구현  
  
(규칙)  
테스트할 때는 이름이 test로 시작하는 메서드를 만들고  
그 안에 테스트할 내용을 적음  
아래 테스트는 square 함수의 출력과 기댓값이 같은지 확인  
self.assertEqual 메스드는 함수의 출력과 기댓값, 두 객체가 동일한지 여부를 판정함  

In [56]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

In [55]:
# test할 파일 저장(test1.py)

with open('test1.py', 'w') as f:
    f.write("""
import unittest
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None: # 데이터가 None이 아니고
            if not isinstance(data, np.ndarray): # ndarray 인스턴스도 아니면
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) # TypeError라는 예외 발생
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) 
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()
        
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
""")

테스트 코드가 test1.py 파일에 있다면  
터미널에서 다음 명령을 실행  
  
\$ python -m unittest test1.py  
  
-----  
  
(python 명령을 실행할 때 앞의 예처럼 -m unittest 인수를 제공하면 파이썬 파일을 테스트 모드로 실행할 수 있음. 혹은 steps10.py 파일 끝에  

~~~python
# test1.py
unittest.main()
~~~
  
위 코드를 추가하면 python test1.py만 입력해도 테스트를 수행할 수 있음)  
  
-----  
  

Ran 1 test in 0.000s   
   
OK   

위와 같이 2 줄이 출력됨  
이는 '1개의 테스트를 실행했고, 결과는 OK다'라는 뜻

이어서 square 함수의 역전파도 테스트하기 위해  
SquareTest 클래스에 다음 코드를 추가

In [57]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

In [58]:
# test할 파일 저장(test2.py)

with open('test2.py', 'w') as f:
    f.write("""
import unittest
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None: # 데이터가 None이 아니고
            if not isinstance(data, np.ndarray): # ndarray 인스턴스도 아니면
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) # TypeError라는 예외 발생
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()
        
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
""")

이처럼 코드를 수정할 때마다 즉시 테스트를 실행해주면 함수의 상태를 반복해서 확인할 수 있음

앞에서는 역전파 테스트할 때 미분의 기댓값을 손으로 계산해 입력함  
이 부분은 기울기 확인(gradient checking)이라는 방법을 통해 자동화할 수 있음  
기울기확인이란 수치 미분으로 구한 결과와 역전파로 구한 결과를 비교하여 차이가 크면 구현에 문제가 있다고 판단하는 검증 기법  
  
4단계에서 수치 미분을 구하기 위해 구현한 numerical_diff 함수를 사용하고  
테스트에 함수 추가

In [59]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):
        x = Variable(np.random.randn(1)) # 무작위 입력값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x) # 수치 미분
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

여기서는 np.allclose 함수 사용  
np.allclose(a,b)는 ndarray 인스턴스인 a와 b의 값이 얼마나 가까운지 판정함  
'얼마나'에 해당하는건 np.allclose(a, b, rtol=1e-5, atol=1e-8)과 같이  
인수 rtol과 atol로 지정할 수 있음  
  
$|a-b|\leq (atol+rtol *|b|)$  
  
위 식의 조건을 만족하면 True를 반환함  
함수에 따라 atol과 rtol의 값을 미세하게 조정해야 할 수도 있음  

In [60]:
# test할 파일 저장(test3.py)

with open('test3.py', 'w') as f:
    f.write("""
import unittest
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None: # 데이터가 None이 아니고
            if not isinstance(data, np.ndarray): # ndarray 인스턴스도 아니면
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data))) # TypeError라는 예외 발생
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() 
            x, y = f.input, f.output
            x.grad = f.backward(y.grad) 
            
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) 
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(Self, gy):
        raise NotImplementError()
        
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    return Square()(x)
    
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):
        x = Variable(np.random.randn(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
""")

테스트 파일들은 한 장소에 모아 관리하는 것이 일반적  
  
$ python -m unittest discover tests  
  
discover라는 하위 명령을 사용하면 discover 다음에 지정한 디렉터리에서 테스트 파일이 있는지 검색 후  
발견한 모든 파일을 실행함  
기본적으로는 지정한 디렉터리에서 이름이 'test*.py'형태인 파일을 테스트 파일로 인식함(변경 가능)  
따라서 위 명령으로 테스트 파일들을 한꺼번에 실행할 수 있음